In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 데이터 불러오기
train  = pd.read_csv('./data/train.csv')
test  = pd.read_csv('./data/test.csv')

pd.set_option('display.max_columns', None)

In [3]:
train.drop_duplicates(inplace=True, keep='last')

In [4]:
# 값이 너무 커서 log 처리
train['intro_log'] = np.log(train['introelapse'])
train['test_log'] = np.log(train['testelapse'])
train['survey_log'] = np.log(train['surveyelapse'])

test['intro_log'] = np.log(test['introelapse'])
test['test_log'] = np.log(test['testelapse'])
test['survey_log'] = np.log(test['surveyelapse'])

In [5]:
train.drop(['country', 'index'], axis=1, inplace=True)
test.drop(['country', 'index'], axis=1, inplace=True)

In [6]:
for i,j in train['age'].iteritems():
    if j>100:
        train.loc[i, 'age'] = 100
        
for i,j in test['age'].iteritems():
    if j>100:
        test.loc[i, 'age'] = 100
        
for i,j in train['familysize'].iteritems():
    if j>10:
        train.loc[i, 'familysize'] = 10
        
for i,j in test['familysize'].iteritems():
    if j>10:
        test.loc[i, 'familysize'] = 10

In [7]:
train.isnull().sum().sum(), test.isnull().sum().sum()

(3513, 7731)

In [8]:
train = train.fillna(method='ffill')

In [9]:
train.isnull().sum().sum(), test.isnull().sum().sum()

(0, 7731)

In [10]:
test.replace([np.inf, -np.inf], np.nan, inplace=True)

In [11]:
train.isnull().sum().sum(), test.isnull().sum().sum()

(0, 7733)

In [12]:
test = test.fillna(method='ffill')

In [13]:
train.isnull().sum().sum(), test.isnull().sum().sum()

(0, 0)

In [14]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.fit(test)

In [17]:
x_train = train.loc[:, 'Q1':'ASD']
y_train = train['nerdiness']

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf_model=RandomForestClassifier(min_samples_split=3,min_samples_leaf=1,max_depth=160,max_features='auto')

In [19]:
from sklearn.model_selection import KFold, GridSearchCV #하이퍼파라미터 튜닝을 위한 불러오기
from sklearn.metrics import accuracy_score

In [20]:
params = { 
    'n_estimators': [200, 500, 1000, 3000, 5000],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

In [21]:
model_gs = GridSearchCV(rf_model, param_grid=params, scoring='accuracy', n_jobs=-1, verbose=1)
model_gs.fit(x_train, y_train)

Fitting 5 folds for each of 150 candidates, totalling 750 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 24.7min


KeyboardInterrupt: 